In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split

df = pd.read_csv("../data/raw/your_dataset.csv") 


df.drop_duplicates(inplace=True)

for col in df.select_dtypes(include='number').columns:
    df[col].fillna(df[col].median(), inplace=True)

for col in df.select_dtypes(include='object').columns:
    df[col].fillna(df[col].mode()[0], inplace=True)


if 'age' in df.columns:
    df["is_adult"] = df["age"].apply(lambda x: 1 if x >= 18 else 0)

if 'date' in df.columns:
    df['date'] = pd.to_datetime(df['date'])
    df['day'] = df['date'].dt.day
    df['month'] = df['date'].dt.month
    df['year'] = df['date'].dt.year


categorical_cols = df.select_dtypes(include=['object']).columns.tolist()


label_enc = LabelEncoder()
for col in categorical_cols:
    if df[col].nunique() == 2:
        df[col] = label_enc.fit_transform(df[col])


df = pd.get_dummies(df, columns=[col for col in categorical_cols if df[col].nunique() > 2], drop_first=True)


scaler = StandardScaler()
numeric_cols = df.select_dtypes(include=['int64', 'float64']).columns.tolist()


target = 'target_column_name'  
if target in numeric_cols:
    numeric_cols.remove(target)

df[numeric_cols] = scaler.fit_transform(df[numeric_cols])


df.to_csv("../data/processed/processed_data.csv", index=False)

print("✅ Feature engineering complete. Processed data saved to data/processed/")